In [2]:
import requests
import pandas as pd
import json

In [3]:
# Suppress SSL warning
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
#### Google Cloud CIDR Block Ranges #########
#https://www.gstatic.com/ipranges/goog.json
#https://www.gstatic.com/ipranges/cloud.json
#https://github.com/ExtraHop/code-examples/blob/main/sunburst/threats.json

In [64]:
import io
from netaddr import IPAddress 
from netaddr import IPNetwork
endpoint = 'https://www.gstatic.com/ipranges/cloud.json'
r = requests.get(endpoint,  verify=False)
data = r.json()['prefixes']
#data = [json.loads(line) for line in open(r, 'r')]
pd.set_option('display.max_rows', None)
df = pd.DataFrame(data)
cidrdf = df['ipv4Prefix']
#cidrdf = df[~df['ipv4Prefix'].str.contains('/')]
cidr = cidrdf.dropna()


In [72]:
import csv
for networks in cidr:
    ip = IPNetwork(networks)
    subnets = list(ip.subnet(24))
    #print subnets
    count = 0
    
    while count < len(subnets):
        #print(subnets[count])
        #count +=1
        with open(r'/Users/johnsmith/notebooks/gCloudcidrs.csv', "ab") as f:  # <-- Your path may vary. 
            writer = csv.writer(f)
            #writer = csv.writer(open("docidrs.csv", "wb"),delimiter='\n')
            writer.writerow([subnets[count]])
            #print subnets[count]
            count +=1

In [69]:
googleData = pd.read_csv(r'/Users/johnsmith/notebooks/gCloudcidrs.csv',names="A")
googleCidr = googleData.loc[:,'A'].str.rsplit(pat='.',n=1,expand=True)[0] # < -- Count by /24 CIDR block
#googleCidr

In [73]:
from parsebrologs import ParseBroLogs
data = ParseBroLogs('conn.log')
df = pd.DataFrame(json.loads(data.to_json()))
#df
#server = df[~df['id.resp_h'].str.contains(':')]
#addr = server['id.resp_h']
cidr = df.loc[:,'id.resp_h'].str.rsplit(pat='.',n=1,expand=True)[0] # < -- Count by /24 CIDR block
cidr.value_counts().head(20)

In [77]:
joined_ips=pd.merge(df,googleData, right_on=googleCidr, left_on=cidr)
joined_df = joined_ips
joined_df
#joined_df.groupby(['id.orig_h','key_0']).size()